# Stock Analysis Case 2

## Business Problem

Working with the same dataset as in case 1,
* find all stocks above their 200 day moving average
* find all stocks below their 200 day moving average
* calculate 50 day moving average for each stock and see how it relates to 200 day moving average
* is there correlation between the two and the direction a stock moves?
* choose 6 or 7 stocks that seem interesting to buy and hold for a month  

## Part 1: Processing files: calculations, identifying high and low stocks, writing to dataframes

#### Goal:
* For each stock, calculate running 200 day and 50 day averages.
* Find stocks that are currently above their running 200 day average, and those currently below.  Save into appropriate dataframes.
* Also calculate crosses (points where 50 day average crosses 200 day average), days where the 50 day average is above the 200 day average, and the stock direction for each day.  Write columns for stock direction and whether the 50 day average is above the 200 day average to a dataframe for use in a logistic regression in part 3.


In [1]:
import google.datalab.storage as storage
import pandas as pd
import numpy as np
from io import BytesIO

In [2]:
# setting up to read data from bucket: filelist returns a list with an object for each file in the bucket 

mybucket = storage.Bucket('st0ckbucket')
filelist = list(mybucket.objects())

In [3]:
# create empty dataframes to populate during loop

high_stocks = pd.DataFrame(columns=['Stock', 'Last_Adj_Close_Price', '200d_Running_Avg', '50d_Running_Avg'])
low_stocks = pd.DataFrame(columns=['Stock', 'Last_Adj_Close_Price', '200d_Running_Avg', '50d_Running_Avg'])
even_stocks = pd.DataFrame(columns=['Stock', 'Last_Adj_Close_Price', '200d_Running_Avg', '50d_Running_Avg'])

more_analysis_df = pd.DataFrame(columns=['50_beats_200', 'up'])

In [4]:
# filelist = filelist[7:8] # toggle on to work with only one file

In [5]:
for file in filelist:
  
  # get filename, stock ticker name, open file, read into df
  filename = str(file)[45:]
  data_csv = mybucket.object(filename)
  data = data_csv.read_stream()
  df = pd.read_csv(BytesIO(data))
  ticker = filename[13:-4]
  
  # skip empty files
  isempty = df.empty
  if isempty:
    continue
    
  # skip files with less then 200 rows-- 200 day running average can't be calculated
  if len(df) < 200:
    continue
    
  print("\n\n Currently working on ", ticker) # toggle for logging
  
  # flip the df because it's currently in desc order, we want asc to compute running averages
  df.sort_values(by=['date'], inplace=True)
  
  # compute running averages, add to df
  df['200d_rolling_avg'] = df.adjclose.rolling(200, min_periods=1).mean()
  df['50d_rolling_avg'] = df.adjclose.rolling(50,min_periods=1).mean()

  # compare 50 & 200, find stock direction
  df['50_beats_200'] = df['50d_rolling_avg'] > df['200d_rolling_avg']
  df['cross'] = df['50_beats_200'].shift() != df['50_beats_200']
  df['up'] = df['adjclose'] > df['adjclose'].shift()

  # grab the latest value, grab adjclose and 200d_rolling_avg
  last = df.tail(1)
  last_price = last['adjclose'][0]
  last_average = last['200d_rolling_avg'][0]

  # determine if last value is higher or lower then 200 day running average, write to appropriate dataframe
  if last_price > last_average:
    high_stocks = high_stocks.append({'Stock': ticker, 'Last_Adj_Close_Price': last_price, '200d_Running_Avg': last_average, '50d_Running_Avg': last['50d_rolling_avg'][0]},  ignore_index = True)
  elif last_price < last_average:
    low_stocks = low_stocks.append({'Stock': ticker, 'Last_Adj_Close_Price': last_price, '200d_Running_Avg': last_average, '50d_Running_Avg': last['50d_rolling_avg'][0]},  ignore_index = True)
  else:
    even_stocks = even_stocks.append({'Stock': ticker, 'Last_Adj_Close_Price': last_price, '200d_Running_Avg': last_average, '50d_Running_Avg': last['50d_rolling_avg'][0]},  ignore_index = True)
    
  # write stock direction column & 50 day average > 200 day average to df for use in part 3's linear regression
  new_df = df[['50_beats_200', 'up']]
  more_analysis_df = more_analysis_df.append(new_df)
  
  print("more_analysis_df has length ", len(more_analysis_df)) # toggle on or off to log size of cumulative dataframe
  
  # delete dataframe to ensure each file is treated fresh
  del df



 Currently working on  AAAU
more_analysis_df has length  474


 Currently working on  AACG
more_analysis_df has length  807


 Currently working on  AADR
more_analysis_df has length  3010


 Currently working on  AAL
more_analysis_df has length  6727


 Currently working on  AAMC
more_analysis_df has length  8523


 Currently working on  AAME
more_analysis_df has length  17744


 Currently working on  AAN
more_analysis_df has length  25529


 Currently working on  AAOI
more_analysis_df has length  27224


 Currently working on  AAON
more_analysis_df has length  34137


 Currently working on  AAP
more_analysis_df has length  38817


 Currently working on  AAPL
more_analysis_df has length  48790


 Currently working on  AAT
more_analysis_df has length  51173


 Currently working on  AAU
more_analysis_df has length  54831


 Currently working on  AAWW
more_analysis_df has length  58511


 Currently working on  AAXJ
more_analysis_df has length  61500


 Currently working on  AB
more_anal

more_analysis_df has length  499252


 Currently working on  AGEN
more_analysis_df has length  504385


 Currently working on  AGFS
more_analysis_df has length  505749


 Currently working on  AGGP
more_analysis_df has length  506793


 Currently working on  AGGY
more_analysis_df has length  507964


 Currently working on  AGI
more_analysis_df has length  510807


 Currently working on  AGIO
more_analysis_df has length  512556


 Currently working on  AGLE
more_analysis_df has length  513624


 Currently working on  AGM-A
more_analysis_df has length  517336


 Currently working on  AGM
more_analysis_df has length  523419


 Currently working on  AGMH
more_analysis_df has length  523973


 Currently working on  AGNC
more_analysis_df has length  527028


 Currently working on  AGNCN
more_analysis_df has length  527753


 Currently working on  AGND
more_analysis_df has length  529133


 Currently working on  AGQ
more_analysis_df has length  532047


 Currently working on  AGR
more_analysi



 Currently working on  AMKR
more_analysis_df has length  931333


 Currently working on  AMLP
more_analysis_df has length  933814


 Currently working on  AMN
more_analysis_df has length  938505


 Currently working on  AMNB
more_analysis_df has length  943576


 Currently working on  AMOT
more_analysis_df has length  952542


 Currently working on  AMOV
more_analysis_df has length  957388


 Currently working on  AMP
more_analysis_df has length  961104


 Currently working on  AMPE
more_analysis_df has length  963642


 Currently working on  AMPH
more_analysis_df has length  965159


 Currently working on  AMPY
more_analysis_df has length  967217


 Currently working on  AMRB
more_analysis_df has length  972007


 Currently working on  AMRC
more_analysis_df has length  974512


 Currently working on  AMRH
more_analysis_df has length  975238


 Currently working on  AMRK
more_analysis_df has length  976813


 Currently working on  AMRN
more_analysis_df has length  982808


 Currently



 Currently working on  ARR
more_analysis_df has length  1352520


 Currently working on  ARTL
more_analysis_df has length  1352897


 Currently working on  ARTNA
more_analysis_df has length  1358943


 Currently working on  ARTW
more_analysis_df has length  1366476


 Currently working on  ARVN
more_analysis_df has length  1366920


 Currently working on  ARW
more_analysis_df has length  1377076


 Currently working on  ARWR
more_analysis_df has length  1381160


 Currently working on  ARYA
more_analysis_df has length  1381408


 Currently working on  ARYAU
more_analysis_df has length  1381663


 Currently working on  ASA
more_analysis_df has length  1392382


 Currently working on  ASC
more_analysis_df has length  1394125


 Currently working on  ASEA
more_analysis_df has length  1396435


 Currently working on  ASET
more_analysis_df has length  1397394


 Currently working on  ASFI
more_analysis_df has length  1403249


 Currently working on  ASG
more_analysis_df has length  141190



 Currently working on  AZZ
more_analysis_df has length  1867232


 Currently working on  B
more_analysis_df has length  1877288


 Currently working on  BA
more_analysis_df has length  1890028


 Currently working on  BAB
more_analysis_df has length  1892702


 Currently working on  BABA
more_analysis_df has length  1894159


 Currently working on  BAC
more_analysis_df has length  1906101


 Currently working on  BAF
more_analysis_df has length  1910549


 Currently working on  BAH
more_analysis_df has length  1912970


 Currently working on  BAK
more_analysis_df has length  1914990


 Currently working on  BAL
more_analysis_df has length  1917956


 Currently working on  BAM
more_analysis_df has length  1927109


 Currently working on  BAND
more_analysis_df has length  1927774


 Currently working on  BANF
more_analysis_df has length  1934901


 Currently working on  BANFP
more_analysis_df has length  1937089


 Currently working on  BANR
more_analysis_df has length  1943280


 Curr

...

more_analysis_df has length  22103524


 Currently working on  UTL
more_analysis_df has length  22111478


 Currently working on  UTMD
more_analysis_df has length  22116755


 Currently working on  UTRN
more_analysis_df has length  22117191


 Currently working on  UTSI
more_analysis_df has length  22122307


 Currently working on  UTSL
more_analysis_df has length  22123072


 Currently working on  UUP
more_analysis_df has length  22126431


 Currently working on  UUU
more_analysis_df has length  22134382


 Currently working on  UUUU
more_analysis_df has length  22137384


 Currently working on  UVSP
more_analysis_df has length  22142563


 Currently working on  UVV
more_analysis_df has length  22154460


 Currently working on  UVXY
more_analysis_df has length  22156358


 Currently working on  UWM
more_analysis_df has length  22159742


 Currently working on  UXI
more_analysis_df has length  22162969


 Currently working on  UXIN
more_analysis_df has length  22163477


 Currently wor

more_analysis_df has length  22517629


 Currently working on  VMC
more_analysis_df has length  22529491


 Currently working on  VMD
more_analysis_df has length  22529709


 Currently working on  VMI
more_analysis_df has length  22541261


 Currently working on  VMO
more_analysis_df has length  22548335


 Currently working on  VMOT
more_analysis_df has length  22549132


 Currently working on  VMW
more_analysis_df has length  22552376


 Currently working on  VNCE
more_analysis_df has length  22554039


 Currently working on  VNDA
more_analysis_df has length  22557610


 Currently working on  VNE
more_analysis_df has length  22560066


 Currently working on  VNET
more_analysis_df has length  22562381


 Currently working on  VNLA
more_analysis_df has length  22563284


 Currently working on  VNM
more_analysis_df has length  22566016


 Currently working on  VNO
more_analysis_df has length  22575714


 Currently working on  VNOM
more_analysis_df has length  22577236


 Currently worki

more_analysis_df has length  22902339


 Currently working on  WBS
more_analysis_df has length  22910713


 Currently working on  WBT
more_analysis_df has length  22911802


 Currently working on  WCC
more_analysis_df has length  22917123


 Currently working on  WCHN
more_analysis_df has length  22917731


 Currently working on  WCN
more_analysis_df has length  22923294


 Currently working on  WDAY
more_analysis_df has length  22925236


 Currently working on  WDC
more_analysis_df has length  22935741


 Currently working on  WDFC
more_analysis_df has length  22947639


 Currently working on  WDIV
more_analysis_df has length  22949408


 Currently working on  WDR
more_analysis_df has length  22955028


 Currently working on  WEA
more_analysis_df has length  22959628


 Currently working on  WEAT
more_analysis_df has length  22961833


 Currently working on  WEBK
more_analysis_df has length  22963406


 Currently working on  WEC
more_analysis_df has length  22973559


 Currently worki

more_analysis_df has length  23456640


 Currently working on  XAN
more_analysis_df has length  23460263


 Currently working on  XAR
more_analysis_df has length  23462343


 Currently working on  XBI
more_analysis_df has length  23465970


 Currently working on  XBIO
more_analysis_df has length  23466875


 Currently working on  XBIT
more_analysis_df has length  23468190


 Currently working on  XBUY
more_analysis_df has length  23468533


 Currently working on  XCEM
more_analysis_df has length  23469541


 Currently working on  XCOM
more_analysis_df has length  23469829


 Currently working on  XDIV
more_analysis_df has length  23470092


 Currently working on  XEC
more_analysis_df has length  23474566


 Currently working on  XEL
more_analysis_df has length  23486512


 Currently working on  XELA
more_analysis_df has length  23487641


 Currently working on  XELB
more_analysis_df has length  23488964


 Currently working on  XENE
more_analysis_df has length  23490379


 Currently wo

more_analysis_df has length  23791262


 Currently working on  ZIXI
more_analysis_df has length  23798983


 Currently working on  ZKIN
more_analysis_df has length  23799696


 Currently working on  ZLAB
more_analysis_df has length  23800397


 Currently working on  ZM
more_analysis_df has length  23800702


 Currently working on  ZN
more_analysis_df has length  23804101


 Currently working on  ZNH
more_analysis_df has length  23809806


 Currently working on  ZOM
more_analysis_df has length  23810463


 Currently working on  ZROZ
more_analysis_df has length  23813123


 Currently working on  ZS
more_analysis_df has length  23813702


 Currently working on  ZSAN
more_analysis_df has length  23814989


 Currently working on  ZSL
more_analysis_df has length  23817863


 Currently working on  ZTO
more_analysis_df has length  23818789


 Currently working on  ZTR
more_analysis_df has length  23826795


 Currently working on  ZTS
more_analysis_df has length  23828663


 Currently working o

In [6]:
more_analysis_df.head()

,50_beats_200,up
473,False,False
472,False,False
471,False,True
470,False,True
469,False,True


In [7]:
len(more_analysis_df)

23835211

In [8]:
high_stocks.head()

,Stock,Last_Adj_Close_Price,200d_Running_Avg,50d_Running_Avg
0,AAAU,17.709999,15.873150,17.242400
1,AACG,1.310000,1.265000,0.921600
2,AADR,54.090000,49.414148,48.995000
3,AAMC,17.400000,14.469300,16.487200
4,AAON,53.730000,50.118015,51.466065


In [9]:
low_stocks.head()

,Stock,Last_Adj_Close_Price,200d_Running_Avg,50d_Running_Avg
0,AAL,12.50,21.311617,12.498000
1,AAME,1.70,2.097400,1.820400
2,AAN,44.73,48.127072,37.717757
3,AAOI,9.80,10.227425,9.260900
4,AAT,28.09,37.968098,27.801254


In [10]:
even_stocks.head()

,Stock,Last_Adj_Close_Price,200d_Running_Avg,50d_Running_Avg


In [11]:
len(high_stocks)

2754

In [12]:
len(low_stocks)

4012

## Part 2: Choosing 6 stocks to buy and hold for a month

We were asked to choose 6 stocks to buy and hold for a month.  We decided to look at the stocks that are currently beneath their 200 day average-- meaning they are low compared to their historical value-- and above their 50 day running average-- meaning that they are high compared to their values over the last 50 days.

In [13]:
# look at the dataframe we already produced in our loop for stocks that are historically low
low_stocks.head()

,Stock,Last_Adj_Close_Price,200d_Running_Avg,50d_Running_Avg
0,AAL,12.50,21.311617,12.498000
1,AAME,1.70,2.097400,1.820400
2,AAN,44.73,48.127072,37.717757
3,AAOI,9.80,10.227425,9.260900
4,AAT,28.09,37.968098,27.801254


In [14]:
# add a new column capturing whether these stocks are higher then their 50 day running average
low_stocks['Above_50d_avg'] = low_stocks['Last_Adj_Close_Price'] > low_stocks['50d_Running_Avg']

In [15]:
# sort by this new column and view the results
low_stocks.sort_values(by=['Above_50d_avg'], inplace=True, ascending=False)
low_stocks.head(10)

,Stock,Last_Adj_Close_Price,200d_Running_Avg,50d_Running_Avg,Above_50d_avg
0,AAL,12.500000,21.311617,12.498000,True
2451,NJV,12.640000,13.201838,12.554954,True
2322,MRIN,1.620000,1.663550,1.499200,True
2323,MRK,78.779999,81.282633,78.038632,True
2326,MRO,6.120000,9.125400,6.012800,True
2328,MSB,17.400000,18.455979,16.377671,True
2329,MSBF,12.000000,14.314313,11.712560,True
2332,MSD,8.540000,8.684368,8.155791,True
2337,MSON,13.440000,15.650800,12.431200,True
2341,MT,10.940000,13.476150,10.287400,True


We decided to choose six stocks that are currently beneath their 200 day running average-- meaning they are historically low-- but above the 50 day running average meaning that they are high compared to the last 50 days.  Therefore, we chose VMC, FID, FGD, EML,FHI and FHK.

## Part 3: Identifying the relationship between 50 day and 200 day running averages, looking for connections with stock direction

In the loop we created a dataframe with two categorical variables: 
* 50_beats_200 is the relationship between the two running averages, True if the 50 day average is higher
* up is the direction of the stock, True if the stock has increased value since the day before

We will run a logistic regression attempting to predict stock direction (up) based on the predictor value 50_beats_20.

In [16]:
# view the dataframe produced in our loop 
# 50_beats_200 is True if the 50 day running average is above the 200 day running average
# Up is true if the stock is higher then it was yesterday.

more_analysis_df.head()

,50_beats_200,up
473,False,False
472,False,False
471,False,True
470,False,True
469,False,True


In [17]:
len(more_analysis_df)
# check the length of the dataframe; we have quite a large dataset

23835211

In [18]:
# prepare the data for logistic regression
# need reshape because we only have one predictor value, logistic regression generally expects more then one

X = more_analysis_df['50_beats_200'].reshape(-1, 1)
y = more_analysis_df['up']

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [19]:
# needs additional processing to prepare for logistic regression
y = np.array(y.values.tolist())

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [21]:
# create the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()

In [22]:
# fit the model
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
# run prediction sand evaluate accuracy of the model
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.53


Based on the low accuracy of the logistic regression (0.53), this is not a good way to predict direction of stock by whether or not the 50 day average is above the 200 day average.

## Next steps:
I suspect that the reason our logistic regression is not successful is that our data is too noisy.  By calculating stock direction based on a single day-- whether one day's value is higher then the previous day-- I have probably introduced too much noise to find correlation.

Next steps would be to smooth the data.  I would first attempt to calculate stock direction looking at a larger number of dates-- perhaps using a running average, perhaps using more manual calculations.  Determining the factor to use for smoothing would be tricky-- should I look at average over a week?  A month?  For this decision,working with several individual files and plotting graphs would probably help determine the right scale of smoothing.

I'd be interested to see if it is possible to create a more accurate logistic regression by filtering out daily fluctuations in this way.